In [1]:
pip install sounddevice

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install sounddevice

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import csv
import queue
import sounddevice as sd
from vosk import Model, KaldiRecognizer
import sys
import json
    

In [4]:
# list all audio devices known to your system
print("Display input/output devices")
print(sd.query_devices())

Display input/output devices
   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Microphone Array (Intel® Smart , MME (2 in, 0 out)
   2 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  3 Speaker (Realtek(R) Audio), MME (0 in, 2 out)
   4 Primary Sound Capture Driver, Windows DirectSound (2 in, 0 out)
   5 Microphone Array (Intel® Smart Sound Technology for Digital Microphones), Windows DirectSound (2 in, 0 out)
   6 Primary Sound Driver, Windows DirectSound (0 in, 2 out)
   7 Speaker (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
   8 Speaker (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
   9 Microphone Array (Intel® Smart Sound Technology for Digital Microphones), Windows WASAPI (2 in, 0 out)
  10 Stereo Mix (Realtek HD Audio Stereo input), Windows WDM-KS (2 in, 0 out)
  11 Headphones 1 (Realtek HD Audio 2nd output with SST), Windows WDM-KS (0 in, 2 out)
  12 Headphones 2 (Realtek HD Audio 2nd output with SST), Windows WDM-KS (0 in, 2 out)
  13 PC Speaker (Real

In [5]:
# get the samplerate - this is needed by the Kaldi recognizer
device_info = sd.query_devices(sd.default.device[0], 'input')
samplerate = int(device_info['default_samplerate'])

# display the default input device
print("===> Initial Default Device Number:{} Description: {}".format(sd.default.device[0], device_info))

===> Initial Default Device Number:1 Description: {'name': 'Microphone Array (Intel® Smart ', 'index': 1, 'hostapi': 0, 'max_input_channels': 2, 'max_output_channels': 0, 'default_low_input_latency': 0.09, 'default_low_output_latency': 0.09, 'default_high_input_latency': 0.18, 'default_high_output_latency': 0.18, 'default_samplerate': 44100.0}


In [6]:
q = queue.Queue()

def recordCallback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    q.put(bytes(indata))

In [33]:
# build the model and recognizer objects.
print("===> Build the model and recognizer objects.  This will take a few minutes.")
model = Model(r"C:\Users\HP\Downloads\vosk-model-small-en-us-0.15\vosk-model-small-en-us-0.15")
recognizer = KaldiRecognizer(model, samplerate)
recognizer.SetWords(False)

print("===> Begin recording. Press Ctrl+C to stop the recording ")
try:
    with sd.RawInputStream(dtype='int16',
                           channels=1,
                           callback=recordCallback):
        while True:
           
            data = q.get()        
            if recognizer.AcceptWaveform(data):
                recognizerResult = recognizer.Result()
                # convert the recognizerResult string into a dictionary  
                resultDict = json.loads(recognizerResult)
                print(resultDict['text'])
                if not resultDict.get("text", "") == "":
                    f=open("sentiment1.csv",'a')
                    print(resultDict['text'],file=f)
                    f.close()
                   
                else:
                    print("no input sound")

except KeyboardInterrupt:
    print('===> Finished Recording')
except Exception as e:
    print(str(e))

===> Build the model and recognizer objects.  This will take a few minutes.
===> Begin recording. Press Ctrl+C to stop the recording 
we have chosen dare not the most famous speeches that ends from celebrated while any crisis
he remembered this celebrated with the next day may have had to give a speech in public
scroll down to the end of disposed for what essential tips
i have a dream to the
i have it

no input sound
wish should wish ill defined our iceland whatever the cost merely
we should fight on the beaches we still fight on the landing jones
which i do not for the children body lavender
we shall never surrender and even if which a to not for the moment believe
i made myself we take up arms and myself when we were generally does and any word of everyone of to barked at in the field
===> Finished Recording


In [34]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]    |   Package basqu

True

In [35]:
df=pd.read_csv("sentiment1.csv",sep="\t",names=["comment_name"])

In [36]:
df.head(10)

,comment_name
0,we have chosen dare not the most famous speech...
1,he remembered this celebrated with the next da...
2,scroll down to the end of disposed for what es...
3,i have a dream to the
4,i have it
5,wish should wish ill defined our iceland whate...
6,we should fight on the beaches we still fight ...
7,which i do not for the children body lavender
8,we shall never surrender and even if which a t...
9,i made myself we take up arms and myself when ...


In [37]:
example=df['comment_name']
# print(example)

series = pd.Series(example)
print(series)

0    we have chosen dare not the most famous speech...
1    he remembered this celebrated with the next da...
2    scroll down to the end of disposed for what es...
3                                i have a dream to the
4                                            i have it
5    wish should wish ill defined our iceland whate...
6    we should fight on the beaches we still fight ...
7        which i do not for the children body lavender
8    we shall never surrender and even if which a t...
9    i made myself we take up arms and myself when ...
Name: comment_name, dtype: object


In [38]:
series_lower = series.str.lower()
print(series_lower)

0    we have chosen dare not the most famous speech...
1    he remembered this celebrated with the next da...
2    scroll down to the end of disposed for what es...
3                                i have a dream to the
4                                            i have it
5    wish should wish ill defined our iceland whate...
6    we should fight on the beaches we still fight ...
7        which i do not for the children body lavender
8    we shall never surrender and even if which a t...
9    i made myself we take up arms and myself when ...
Name: comment_name, dtype: object


In [39]:
a_string = (str(series_lower))
new_string = re.sub(r'[^\w\s]', '', a_string)

print(new_string)

0    we have chosen dare not the most famous speech
1    he remembered this celebrated with the next da
2    scroll down to the end of disposed for what es
3                                i have a dream to the
4                                            i have it
5    wish should wish ill defined our iceland whate
6    we should fight on the beaches we still fight 
7        which i do not for the children body lavender
8    we shall never surrender and even if which a t
9    i made myself we take up arms and myself when 
Name comment_name dtype object


In [40]:
space_pattern = (str(new_string))
remove_extra_spaces = re.sub( r'\s+', repl=' ', string=space_pattern)
print(space_pattern)

0    we have chosen dare not the most famous speech
1    he remembered this celebrated with the next da
2    scroll down to the end of disposed for what es
3                                i have a dream to the
4                                            i have it
5    wish should wish ill defined our iceland whate
6    we should fight on the beaches we still fight 
7        which i do not for the children body lavender
8    we shall never surrender and even if which a t
9    i made myself we take up arms and myself when 
Name comment_name dtype object


In [41]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
sentence = " ".join([word for word in str(space_pattern).split() if word not in STOPWORDS])
print(sentence)

0 chosen dare famous speech 1 remembered celebrated next da 2 scroll end disposed es 3 dream 4 5 wish wish ill defined iceland whate 6 fight beaches still fight 7 children body lavender 8 shall never surrender even 9 made take arms Name comment_name dtype object


In [42]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize(sentence))

0 chosen dare famous speech 1 remembered celebrated next da 2 scroll end disposed es 3 dream 4 5 wish wish ill defined iceland whate 6 fight beaches still fight 7 children body lavender 8 shall never surrender even 9 made take arms Name comment_name dtype object


In [47]:
token=nltk.sent_tokenize(sentence)

In [48]:
token

['0 chosen dare famous speech 1 remembered celebrated next da 2 scroll end disposed es 3 dream 4 5 wish wish ill defined iceland whate 6 fight beaches still fight 7 children body lavender 8 shall never surrender even 9 made take arms Name comment_name dtype object']

In [49]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia=SentimentIntensityAnalyzer()

In [50]:
for comment_list in token:
    ss = sia.polarity_scores(comment_list) 
    print(ss)

{'neg': 0.163, 'neu': 0.611, 'pos': 0.226, 'compound': 0.4767}
